# [Calculator I](https://leetcode.com/problems/basic-calculator/)
- Give a string of expression.
    + Contains: (, ), +, -
    + integer
- Evaluate the expression.

```
Input: "1 + 1"
Output: 2

Input: " 2-1 + 2 "
Output: 3

Input: "(1+(4+5+2)-3)+(6+8)"
Output: 23

Input: "59 + 19+(7-8-( 3+2)+3)"
Output: 75
```


## Solution - Stack
#### Model
- `+num` or `-num`
- `+(<inp_ops>)` or `-(<inp_ops>)`

#### Rules
- digit: Update current number
- '+':
    + number is over
    + Add the previous number
    + start a new positive number
- '-':
    + number is over
    + Add the previous number
    + start a new negative number
- '(': 
    + push the previous result and the sign into the stack,
    + set result to 0
    + calculate the new result within the parenthesis.
- ')': number is over
    + pop out the sign before this pair of parenthesis, and temporary result before this pair of parenthesis
    + Update result

#### Code

```C++
class Solution {
public:
    int calculate(const string &Str) {
        stack<int> S;
        int num = 0;
        int sign = 1;
        int result = 0;
        for(int i=0; i<Str.size(); ++i) {
            // Case: digit 0 - 9
            while('0' <= Str[i] && Str[i] <= '9') {
                num = 10*num + (Str[i] - '0');
                i += 1;
            }


            char op = Str[i];
            // Case: +
            if(op == '+') {
                // Update prev num
                result += sign*num;

                // Start +num
                sign = 1;
                num = 0;
            }

            // Case: -
            else if(op == '-') {
                // Update prev num
                result += sign*num;

                // Start -num
                sign = -1;
                num = 0;
            }

            // Case: (
            else if(op == '(') {
                // Save cur state
                S.push(result); // push res
                S.push(sign);  // push sign

                // start (<ops>)
                result = 0;
                sign = 1;
            }

            else if(op == ')') {
                // Update <ops>
                result += sign*num;

                // Update sign(<ops>)
                result *= S.top(); S.pop();  // pop sign
                result += S.top(); S.pop();  // pop res

                // start new num
                num = 0;
                sign = 1;
            }
        }
        return result + sign*num;
    }
};
```

# [Calculator II](https://leetcode.com/problems/basic-calculator-ii/)
- Give a string of expression.
    + Contains: +,-,*,/
    + integer
- Evaluate the expression.

```
Input: s = "3+2*2"
Output: 7

Input: s = " 3/2 "
Output: 1

Input: "(1+(4+5+2)-3)+(6+8)"
Output: 23

Input: s = " 3+5 / 2 "
Output: 5
```

#### Model
+ `+num1*num2` or `-num1*num2`

#### Code

```C++
class Solution {
public:
    int calculate(const string &s) {
        vector<long> nums;

        char op = '+';
        int idx = 0;
        for(;idx < s.size(); ++idx) {
            if(s[idx] == ' ') continue;

            // parse num
            long num = 0;
            while('0' <= s[idx] && s[idx] <= '9') {
                num = 10*num + (s[idx] - '0');
                idx += 1;
            }

            // Update last op
            if(op == '+') nums.push_back(num);
            else if(op == '-') nums.push_back(-num);
            else if(op == '*') nums.back() *= num;
            else if(op == '/') nums.back() /= num;

            // Reset, Get new op
            if(idx < s.size()) op = s[idx];
        }
        return static_cast<int>(accumulate(nums.begin(), nums.end(), 0));
    }
};
```

# [Calculator III](https://leetcode.com/problems/basic-calculator-iii/)
- Give a string of expression.
    + Contains: (, ), +, -, *, /
    + integer
- Evaluate the expression.

```
Input: "1 + 1"
Output: 2

Input: " 6-4 / 2 "
Output: 4

Input: "2*(5+5*2)/3+(6/2+8)"
Output: 21

Input: "(2+6* 3+5- (3*14/7+2)*5)+3"
Output: -12
```


#### Solution: Recursive
    
```C++
// template
class Solution {
private:
    string _s;
    int _N;
public:
    int __eval(int &idx) {
        vector<int> nums;
        char op = '+';
        for(; idx<_s.size(); ++idx) {
            // Case: ' '
            if (_s[idx] == ' ') continue;

            long num = 0;

            // Case: digit
            while(idx < _N && '0' <= _s[idx] && _s[idx] <= '9') {
                num = 10*num + (_s[idx] - '0');
                idx += 1;
            }

            // Case: '('
            if(_s[idx] == '(') num = __eval(++idx); 

            // Update result from last op
            if(op == ')') return accumulate(nums.begin(), nums.end(), 0);
            else if(op == '+') nums.push_back(num);
            else if(op == '-') nums.push_back(-num);
            else if(op == '*') nums.back() *= num;
            else if(op == '/') nums.back() /= num;

            // update new op: +,-,*,/,)
            if(idx < _s.size()) op = _s[idx];
        }
        return accumulate(nums.begin(), nums.end(), 0);
    }
    int calculate(const string &s) {
        _s = s;
        _N = _s.size();

        // dfs
        int idx = 0;
        return __eval(idx);
    }
};


// optimized clearance
class Solution {
private:
    string _s;
    int _N;
public:
    int __parse_num(int &idx) {
        int num = 0;
        for(; idx < _N && '0' <= _s[idx] && _s[idx] <= '9'; ++idx) {
            num = 10*num + (_s[idx] - '0');
        }
        return num;
    }
    int __eval(int &idx) {
        vector<int> nums;
        char op = '+';
        for(; idx<_s.size(); ++idx) {
            if (_s[idx] == ' ') continue;

            // parse num from next() or cur
            long num = (_s[idx] == '(') ? __eval(++idx) : __parse_num(idx);

            // Update result from last op
            if(op == ')') return accumulate(nums.begin(), nums.end(), 0);
            else if(op == '+') nums.push_back(num);
            else if(op == '-') nums.push_back(-num);
            else if(op == '*') nums.back() *= num;
            else if(op == '/') nums.back() /= num;

            // update new op: +,-,*,/,)
            if(idx < _s.size()) op = _s[idx];
        }
        return accumulate(nums.begin(), nums.end(), 0);
    }
    int calculate(const string &s) {
        _s = s;
        _N = _s.size();

        // dfs
        int idx = 0;
        return __eval(idx);
    }
};
```